In [1]:
import control as ct
import numpy as np

In [ ]:
A = np.array([])
B = np.array([])
C = np.array([])
D = np.array([])

In [ ]:
quad_linear_model = ct.ss(A, B, C, D)
Q = np.diag([])
R = np.diag([])
N = np.array([])

K, S, E = ct.lqr(quad_linear_model, Q, R, N)

print(K)

## Controllability

In [ ]:
C_Matrix = ct.ctrb()
np.linalg.matrix_rank(C_Matrix)
# Controlability Gramian

## Controller Design

In [ ]:
K, S, E = ct.lqr(quad_linear_model, Q, R, N)

## Trajectory Analysis

## Multi-Agent System Analysis

In [ ]:
ct.lyap()